Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.442681
Minibatch accuracy: 18.8%
Validation accuracy: 11.1%
Minibatch loss at step 50: 1.663865
Minibatch accuracy: 37.5%
Validation accuracy: 52.2%
Minibatch loss at step 100: 1.117523
Minibatch accuracy: 75.0%
Validation accuracy: 73.1%
Minibatch loss at step 150: 0.795176
Minibatch accuracy: 81.2%
Validation accuracy: 72.9%
Minibatch loss at step 200: 1.502430
Minibatch accuracy: 56.2%
Validation accuracy: 75.5%
Minibatch loss at step 250: 0.468325
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.816013
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.822662
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.378167
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 450: 1.197121
Minibatch accuracy: 68.8%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.356691
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [33]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

filter_size = patch_size
conv_pad = (filter_size - 1) / 2
conv_stride = 1

pool_width = pool_height = 2
pool_stride = 2


H = image_size
W = image_size
conv1_out_h = 1 + (H + 2 * conv_pad - filter_size) / conv_stride
conv1_out_w = 1 + (W + 2 * conv_pad - filter_size) / conv_stride
pool1_out_w = (conv1_out_w - pool_width) / pool_stride + 1
pool1_out_h = (conv1_out_h - pool_height) / pool_stride + 1


H = pool1_out_h
W = pool1_out_w
conv2_out_h = 1 + (H + 2 * conv_pad - filter_size) / conv_stride
conv2_out_w = 1 + (W + 2 * conv_pad - filter_size) / conv_stride
pool2_out_w = (conv2_out_w - pool_width) / pool_stride + 1
pool2_out_h = (conv2_out_h - pool_height) / pool_stride + 1


# Calc out size for 2x2 Max Pool


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  #layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_weights = tf.Variable(tf.truncated_normal([pool2_out_w*pool2_out_h*depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(relu1, [1, pool_width, pool_height, 1], [1, pool_stride, pool_stride, 1], padding='SAME')
    
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    relu2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(relu2, [1, pool_width, pool_height, 1], [1, pool_stride, pool_stride, 1], padding='SAME')
    
    pool2_shape = pool2.get_shape().as_list()
    pool2_reshaped = tf.reshape(pool2, [pool2_shape[0], pool2_shape[1] * pool2_shape[2] * pool2_shape[3]])
    fully_connected1 = tf.nn.relu(tf.matmul(pool2_reshaped, layer3_weights) + layer3_biases)
    
    return tf.matmul(fully_connected1, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [35]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
    
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 4.648585
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.215277
Minibatch accuracy: 18.8%
Validation accuracy: 18.5%
Minibatch loss at step 100: 1.767671
Minibatch accuracy: 43.8%
Validation accuracy: 52.5%
Minibatch loss at step 150: 0.996031
Minibatch accuracy: 68.8%
Validation accuracy: 69.7%
Minibatch loss at step 200: 1.452278
Minibatch accuracy: 62.5%
Validation accuracy: 73.7%
Minibatch loss at step 250: 0.613447
Minibatch accuracy: 81.2%
Validation accuracy: 75.5%
Minibatch loss at step 300: 0.767588
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.550945
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.607967
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 450: 1.136327
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.273480
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
from cnn import CnnParams, CnnModel, CNN, CnnSolver

input_dim = [image_size, image_size, num_channels]
num_classes = num_labels

In [6]:
# Model params
conv_filter_size = 5#7
conv_num_filters = 16#32
conv_stride = 1
pool_size = 2
pool_stride = 2
num_hidden_fcn = 64#1024


# Solver params
batches_num = 100
batch_size = 1024#train_dataset.shape[0] / batches_num
num_epochs = 15#500
dropout_prob = 0.5

l2 = 1e-3
lr_start = 1e-3
lr_decay_steps = 100000
lr_decay_rate = 0.96
ws = 0.1

cnn_params = CnnParams(input_dim, conv_filter_size, conv_num_filters, conv_stride,
                       pool_size, pool_stride, num_hidden_fcn, num_classes, ws)
model = CnnModel(cnn_params, init_weights=True)
cnn = CNN(model, l2)
solver = CnnSolver(cnn, train_dataset, train_labels, valid_dataset, valid_labels, 
                batch_size, num_epochs, dropout_prob, 
                lr_start, lr_decay_steps, lr_decay_rate)
(valid_loss, valid_accuracy, valid_model) = solver.train()

print("*************** Finished ***************")
print("Best Valid Accuracy: %.1f%% on L2: %e, WeightScale: %e -----------------" % (valid_accuracy*100, l2, ws))

#cnn.update_from_model(valid_model)
#test_accuracy_cnn = cnn
test_accuracy_cnn = CNN(valid_model, l2)
with tf.Session(graph=test_accuracy_cnn.graph) as session:
    session.run(tf.initialize_all_variables())
    test_accuracy = test_accuracy_cnn.accuracy.eval(feed_dict={test_accuracy_cnn.X:test_dataset, 
                                                             test_accuracy_cnn.Y:test_labels, 
                                                             test_accuracy_cnn.dropout: 1.0})
print("*************** Test accuracy: %.1f%% *****************" % (test_accuracy*100))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
------------------- Epoch 0 of 15 ----------------------
Minibatch loss: 4.558469
Minibatch train accuracy: 10.1%
Validation accuracy: 9.7%
------------------- Epoch 1 of 15 ----------------------
Minibatch loss: 1.043974
Minibatch train accuracy: 82.5%
Validation accuracy: 82.1%
------------------- Epoch 2 of 15 ----------------------
Minibatch loss: 0.753011
Minibatch train accuracy: 86.7%
Validation accuracy: 85.5%
------------------- Epoch 3 of 15 ----------------------
Minibatch loss: 0.710201
Minibatch train accuracy: 86.7%
Validation accuracy: 86.5%
------------------- Epoch 4 of 15 ----------------------
Minibatch loss: 0.665690
Minibatch train accuracy: 87.8%
Validation accuracy: 87.6%
------------------- Epoch 5 of 15 ----------------------
Minibatch loss: 0.613626
Minibatch train accuracy: 87.9%
Validation accuracy: 87.8%
------------------- Epoch 6 of 15 ----------------------
Minibatch lo